In [1]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive_output, VBox, HBox, Layout
from scipy.optimize import curve_fit

# Load the experimental image once from the .npy file
exp_image = np.load('/Users/xiaodong/Downloads/glycine_frame4/glycine_frame4.npy')

def pseudo_voigt(xx, yy, x0, y0, amplitude, sigma, gamma, eta):
    """
    Generate a pseudo Voigt profile.
    - Gaussian: exp(-((x-x0)^2+(y-y0)^2)/(2*sigma^2))
    - Lorentzian: 1/(1+((x-x0)^2+(y-y0)^2)/gamma^2)
    - eta: mixing factor (0 => pure Gaussian, 1 => pure Lorentzian)
    """
    r2 = (xx - x0)**2 + (yy - y0)**2
    gaussian = np.exp(-r2 / (2 * sigma**2))
    lorentzian = 1 / (1 + r2 / gamma**2)
    return amplitude * (eta * lorentzian + (1 - eta) * gaussian)

def simulate_image(nx, ny, beam_center_x, beam_center_y, amplitude, sigma, gamma, eta, background, poisson_level, read_noise_std):
    # Create coordinate grids
    x = np.arange(nx)
    y = np.arange(ny)
    xx, yy = np.meshgrid(x, y)
    # Generate simulated beam
    beam = pseudo_voigt(xx, yy, beam_center_x, beam_center_y, amplitude, sigma, gamma, eta)
    # Add noise components
    noise_poisson = np.random.poisson(poisson_level, size=(ny, nx))
    noise_read = np.random.normal(loc=0, scale=read_noise_std, size=(ny, nx))
    image = beam + background + noise_poisson + noise_read
    image[image < 0] = 0  # ensure no negatives
    return image

def radial_profile(data, center, r_min=5):
    """
    Compute the radial profile of a 2D array and mask out data below r_min.
    
    Returns:
      r_i         : Radii (pixels) for which the profile is computed (r >= r_min)
      radial_mean : Average intensity in annular bins (for r >= r_min)
    """
    y, x = np.indices(data.shape)
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2)
    r_int = r.astype(int)
    tbin = np.bincount(r_int.ravel(), data.ravel())
    nr = np.bincount(r_int.ravel())
    radial_mean = tbin / nr
    r_i = np.arange(len(radial_mean))
    mask = r_i >= r_min
    return r_i[mask], radial_mean[mask]

def pseudo_voigt_1d(r, amplitude, sigma, gamma, eta, baseline):
    """
    1D pseudo Voigt function for fitting the radial profile.
    """
    gaussian = np.exp(-r**2 / (2 * sigma**2))
    lorentzian = 1 / (1 + (r / gamma)**2)
    return amplitude * (eta * lorentzian + (1 - eta) * gaussian) + baseline

def update_plots(amplitude, sigma, gamma, eta, background, poisson_level, read_noise_std,
                 beam_center_x, beam_center_y, exp_radial_center_x, exp_radial_center_y):
    nx, ny = 512, 512
    # Generate simulated image
    sim_img = simulate_image(nx, ny, beam_center_x, beam_center_y, amplitude, sigma, gamma, eta,
                             background, poisson_level, read_noise_std)
    
    # Compute radial profiles (for simulation and experimental data)
    sim_center = (beam_center_x, beam_center_y)
    r_sim, profile_sim = radial_profile(sim_img, sim_center)
    exp_center = (exp_radial_center_x, exp_radial_center_y)
    r_exp, profile_exp = radial_profile(exp_image, exp_center)
    
    # Fit the experimental radial profile using pseudo_voigt_1d
    try:
        # Initial guess: amplitude from data range, sigma and gamma ~10, eta 0.5, baseline from min intensity.
        p0 = [max(profile_exp)-min(profile_exp), 10, 10, 0.5, min(profile_exp)]
        popt, pcov = curve_fit(pseudo_voigt_1d, r_exp, profile_exp, p0=p0)
        fitted_curve = pseudo_voigt_1d(r_exp, *popt)
        fit_text = (f"Amplitude: {popt[0]:.2f}\nSigma: {popt[1]:.2f}\n"
                    f"Gamma: {popt[2]:.2f}\nEta: {popt[3]:.2f}\nBaseline: {popt[4]:.2f}")
    except Exception as e:
        popt = None
        fitted_curve = None
        fit_text = "Fit failed."
    
    # Update the radial profile plot
    with out_radial:
        out_radial.clear_output(wait=True)
        fig, ax = plt.subplots(figsize=(6,6))
        ax.plot(r_sim, profile_sim, 'b-', label=f"Simulated\n(center={beam_center_x},{beam_center_y})")
        ax.plot(r_exp, profile_exp, 'r--', label=f"Experimental\n(center={exp_radial_center_x},{exp_radial_center_y})")
        if fitted_curve is not None:
            ax.plot(r_exp, fitted_curve, 'k:', label="Fit")
            ax.text(0.05, 0.95, fit_text, transform=ax.transAxes, fontsize=10,
                    verticalalignment='top', bbox=dict(boxstyle="round", facecolor="wheat", alpha=0.5))
        ax.set_title("Radial Profiles & Fit")
        ax.set_xlabel("Radius (pixels)")
        ax.set_ylabel("Average Intensity")
        ax.legend()
        ax.grid(True)
        plt.show()
    
    # Use same intensity scale for images (from experimental image)
    vmin = exp_image.min()
    vmax = exp_image.max()
    with out_images:
        out_images.clear_output(wait=True)
        fig, axs = plt.subplots(1, 2, figsize=(12,6))
        im0 = axs[0].imshow(sim_img, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
        axs[0].set_title("Simulated Diffraction")
        axs[0].set_xlabel("Pixel (fs)")
        axs[0].set_ylabel("Pixel (ss)")
        fig.colorbar(im0, ax=axs[0], label='Intensity')
        
        im1 = axs[1].imshow(exp_image, cmap='gray', origin='lower', vmin=vmin, vmax=vmax)
        axs[1].set_title("Experimental Background (Frame 4)")
        axs[1].set_xlabel("Pixel (fs)")
        axs[1].set_ylabel("Pixel (ss)")
        fig.colorbar(im1, ax=axs[1], label='Intensity')
        plt.tight_layout()
        plt.show()

# Create slider controls as separate widgets
amp_slider = widgets.IntSlider(value=1000, min=0, max=10000, step=100, description='Amplitude')
sigma_slider = widgets.FloatSlider(value=10, min=1, max=50, step=0.5, description='Sigma')
gamma_slider = widgets.FloatSlider(value=10, min=1, max=50, step=0.5, description='Gamma')
eta_slider = widgets.FloatSlider(value=0.5, min=0, max=1, step=0.01, description='Eta')
background_slider = widgets.IntSlider(value=10, min=0, max=100, step=1, description='Background')
poisson_slider = widgets.IntSlider(value=10, min=0, max=100, step=1, description='Poisson')
read_noise_slider = widgets.IntSlider(value=5, min=0, max=20, step=1, description='Read Noise')
beam_center_x_slider = widgets.IntSlider(value=256, min=0, max=512, step=1, description='Beam Center X')
beam_center_y_slider = widgets.IntSlider(value=256, min=0, max=512, step=1, description='Beam Center Y')
exp_radial_center_x_slider = widgets.IntSlider(value=238, min=206, max=306, step=1, description='Exp Radial Center X')
exp_radial_center_y_slider = widgets.IntSlider(value=242, min=206, max=306, step=1, description='Exp Radial Center Y')

# Output widgets for the radial profile plot and the images plot
out_radial = widgets.Output()
out_images = widgets.Output()

# Arrange the slider controls in a vertical box
controls = VBox([amp_slider, sigma_slider, gamma_slider, eta_slider,
                 background_slider, poisson_slider, read_noise_slider,
                 beam_center_x_slider, beam_center_y_slider,
                 exp_radial_center_x_slider, exp_radial_center_y_slider])

# Link the sliders to the update function using interactive_output
interactive_out = interactive_output(update_plots, {
    'amplitude': amp_slider,
    'sigma': sigma_slider,
    'gamma': gamma_slider,
    'eta': eta_slider,
    'background': background_slider,
    'poisson_level': poisson_slider,
    'read_noise_std': read_noise_slider,
    'beam_center_x': beam_center_x_slider,
    'beam_center_y': beam_center_y_slider,
    'exp_radial_center_x': exp_radial_center_x_slider,
    'exp_radial_center_y': exp_radial_center_y_slider
})

# Layout: Top row has the controls and the radial profile plot side by side; bottom row has the images.
top_row = HBox([controls, out_radial], layout=Layout(align_items='flex-start'))
layout = VBox([top_row, out_images])
display(layout)

# Trigger an initial update
update_plots(amp_slider.value, sigma_slider.value, gamma_slider.value, eta_slider.value,
             background_slider.value, poisson_slider.value, read_noise_slider.value,
             beam_center_x_slider.value, beam_center_y_slider.value,
             exp_radial_center_x_slider.value, exp_radial_center_y_slider.value)
